In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
data0 = pd.read_csv("2.online-valid.csv")
data1=pd.read_csv('1.Benign_list_big_final.csv')

data1.columns = ['URLs']

In [ ]:
phishurl = data0.sample(n = 5000, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
legiurl = data1.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)

In [ ]:
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6514946,http://confirmprofileaccount.com/,http://www.phishtank.com/phish_detail.php?phis...,2020-04-19T11:06:55+00:00,yes,2020-04-19T13:42:41+00:00,yes,Other
1,4927651,http://www.marreme.com/MasterAdmin/04mop.html,http://www.phishtank.com/phish_detail.php?phis...,2017-04-04T19:35:54+00:00,yes,2017-05-03T23:00:42+00:00,yes,Other
2,5116976,http://modsecpaststudents.com/review/,http://www.phishtank.com/phish_detail.php?phis...,2017-07-25T18:48:30+00:00,yes,2017-07-28T16:01:36+00:00,yes,Other
3,6356131,https://docs.google.com/forms/d/e/1FAIpQLScL6L...,http://www.phishtank.com/phish_detail.php?phis...,2020-01-13T20:13:37+00:00,yes,2020-01-17T01:55:38+00:00,yes,Other
4,6535965,https://oportunidadedasemana.com/americanas//?...,http://www.phishtank.com/phish_detail.php?phis...,2020-04-29T00:01:03+00:00,yes,2020-05-01T10:55:35+00:00,yes,Other


In [ ]:
from urllib.parse import urlparse,urlencode
import ipaddress
import re

In [ ]:
def getDomain(url):
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
	       domain = domain.replace("www.","")
  return domain

In [ ]:
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

In [ ]:
def haveAtSign(url):
  if "@" in url:
    at = 1
  else:
    at = 0
  return at

In [ ]:
def getLength(url):
  if len(url) < 54:
    length = 0
  else:
    length = 1
  return length

In [ ]:
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [ ]:
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

In [ ]:
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [ ]:
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"


In [ ]:
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [ ]:
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate

In [ ]:
!pip install python-whois

In [ ]:
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime

In [ ]:
def web_traffic(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 0

In [ ]:
def domainAge(domain_name):

    creation_date = domain_name.creation_date
    expiration_date = domain_name.expiration_date

    if creation_date is None or expiration_date is None:
        return 1

    if isinstance(creation_date, list):
        creation_date = creation_date[0]

    if isinstance(expiration_date, list):
        expiration_date = expiration_date[0]

    if isinstance(creation_date, str):
        try:
            creation_date = datetime.strptime(creation_date, "%Y-%m-%d")
        except:
            return 1

    if isinstance(expiration_date, str):
        try:
            expiration_date = datetime.strptime(expiration_date, "%Y-%m-%d")
        except:
            return 1

    # Remove timezone
    if creation_date.tzinfo is not None:
        creation_date = creation_date.replace(tzinfo=None)

    if expiration_date.tzinfo is not None:
        expiration_date = expiration_date.replace(tzinfo=None)

    age_days = abs((expiration_date - creation_date).days)

    if (age_days / 30) < 6:
        return 1   # suspicious
    else:
        return 0   # safe

In [ ]:
from datetime import datetime, timezone

def domainEnd(domain_name):

    expiration_date = domain_name.expiration_date

    if expiration_date is None:
        return 1

    # If list → take first value
    if isinstance(expiration_date, list):
        expiration_date = expiration_date[0]

    # If string → convert
    if isinstance(expiration_date, str):
        try:
            expiration_date = datetime.strptime(expiration_date, "%Y-%m-%d")
        except:
            return 1

    # Remove timezone if exists
    if expiration_date.tzinfo is not None:
        expiration_date = expiration_date.replace(tzinfo=None)

    today = datetime.now()

    end_days = abs((expiration_date - today).days)

    if (end_days / 30) < 6:
        return 0   # risky (phishing-like)
    else:
        return 1   # safe

In [ ]:
import requests

In [ ]:
def iframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[|]", response.text):
          return 0
      else:
          return 1

In [ ]:
def mouseOver(response):
  if response == "" :
    return 1
  else:
    if re.findall("", response.text):
      return 1
    else:
      return 0

In [ ]:
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

In [ ]:
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

In [ ]:
!pip install tldextract

In [ ]:
# def featureExtraction(url):

#     features = []

#     domain = getDomain(url)

#     try:
#         response = requests.get(url, timeout=5)
#     except:
#         response = ""

#     try:
#         domain_info = whois.whois(domain)
#     except:
#         domain_info = ""

#     # Address bar features
#     features.append(havingIP(url))
#     features.append(haveAtSign(url))
#     features.append(getLength(url))
#     features.append(getDepth(url))
#     features.append(redirection(url))
#     features.append(httpDomain(url))
#     features.append(tinyURL(url))
#     features.append(prefixSuffix(url))

#     # Domain features
#     features.append(domainAge(domain_info))
#     features.append(domainEnd(domain_info))

#     # HTML & JS features
#     features.append(iframe(response))
#     features.append(mouseOver(response))
#     features.append(rightClick(response))
#     features.append(forwarding(response))

#     return features



import tldextract

def featureExtraction(url):

    features = []

    ext = tldextract.extract(url)
    domain = ext.domain
    suffix = ext.suffix

    features.append(len(url))

    features.append(url.count('.'))

    features.append(url.count('-'))

    features.append(url.count('@'))

    features.append(url.count('?'))

    features.append(url.count('%'))

    features.append(url.count('='))

    features.append(url.count('http'))

    features.append(url.count('https'))

    features.append(url.count('www'))

    features.append(sum(c.isdigit() for c in url))

    features.append(sum(c.isalpha() for c in url))

    features.append(tinyURL(url))

    features.append(prefixSuffix(url))

    return features

In [ ]:
legi_features = []
label = 0

for i in range(0, 5000):
  url = legiurl['URLs'][i]
  features = (featureExtraction(url))
  features.append(label)
  legi_features.append(features)

In [ ]:
feature_names = [
    'URL_Length','Dot_Count','Hyphen_Count','At_Count','QMark_Count',
    'Percent_Count','Equal_Count','HTTP_Count','HTTPS_Count','WWW_Count',
    'Digit_Count','Letter_Count','TinyURL','Prefix_Suffix','Label'
]

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()

,URL_Length,Dot_Count,Hyphen_Count,At_Count,QMark_Count,Percent_Count,Equal_Count,HTTP_Count,HTTPS_Count,WWW_Count,Digit_Count,Letter_Count,TinyURL,Prefix_Suffix,Label
0,138,1,1,0,1,0,10,1,0,0,0,107,0,0,0
1,105,4,1,0,2,2,4,2,0,1,22,55,0,0,0
2,89,1,3,0,1,2,2,1,1,0,2,72,0,0,0
3,132,4,0,0,0,5,0,1,0,0,19,85,0,0,0
4,115,2,6,0,1,0,1,1,0,0,17,82,0,0,0


In [ ]:
legitimate.to_csv('legitimate.csv', index= False)

In [ ]:
phish_features = []
label = 1
for i in range(0, 5000):
  url = phishurl['url'][i]
  features = (featureExtraction(url))
  features.append(label)
  phish_features.append(features)

In [ ]:
feature_names = [
    'URL_Length','Dot_Count','Hyphen_Count','At_Count','QMark_Count',
    'Percent_Count','Equal_Count','HTTP_Count','HTTPS_Count','WWW_Count',
    'Digit_Count','Letter_Count','TinyURL','Prefix_Suffix','Label'
]

phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,URL_Length,Dot_Count,Hyphen_Count,At_Count,QMark_Count,Percent_Count,Equal_Count,HTTP_Count,HTTPS_Count,WWW_Count,Digit_Count,Letter_Count,TinyURL,Prefix_Suffix,Label
0,33,1,0,0,0,0,0,1,0,0,0,28,1,0,1
1,45,3,0,0,0,0,0,1,0,1,2,35,0,0,1
2,37,1,0,0,0,0,0,1,0,0,0,31,0,0,1
3,99,2,1,0,0,0,0,1,1,0,4,83,0,0,1
4,166,1,13,0,1,0,2,1,1,0,21,118,0,0,1


In [ ]:

phishing.to_csv('phishing.csv', index= False)

In [ ]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.head()

,URL_Length,Dot_Count,Hyphen_Count,At_Count,QMark_Count,Percent_Count,Equal_Count,HTTP_Count,HTTPS_Count,WWW_Count,Digit_Count,Letter_Count,TinyURL,Prefix_Suffix,Label
0,138,1,1,0,1,0,10,1,0,0,0,107,0,0,0
1,105,4,1,0,2,2,4,2,0,1,22,55,0,0,0
2,89,1,3,0,1,2,2,1,1,0,2,72,0,0,0
3,132,4,0,0,0,5,0,1,0,0,19,85,0,0,0
4,115,2,6,0,1,0,1,1,0,0,17,82,0,0,0


In [ ]:

urldata.to_csv('urldata.csv', index=False)

In [ ]:
df = pd.read_csv("urldata.csv")
df.head()

,URL_Length,Dot_Count,Hyphen_Count,At_Count,QMark_Count,Percent_Count,Equal_Count,HTTP_Count,HTTPS_Count,WWW_Count,Digit_Count,Letter_Count,TinyURL,Prefix_Suffix,Label
0,138,1,1,0,1,0,10,1,0,0,0,107,0,0,0
1,105,4,1,0,2,2,4,2,0,1,22,55,0,0,0
2,89,1,3,0,1,2,2,1,1,0,2,72,0,0,0
3,132,4,0,0,0,5,0,1,0,0,19,85,0,0,0
4,115,2,6,0,1,0,1,1,0,0,17,82,0,0,0


In [ ]:
X = df.drop([ 'Label'],axis=1)
X.head()

,URL_Length,Dot_Count,Hyphen_Count,At_Count,QMark_Count,Percent_Count,Equal_Count,HTTP_Count,HTTPS_Count,WWW_Count,Digit_Count,Letter_Count,TinyURL,Prefix_Suffix
0,138,1,1,0,1,0,10,1,0,0,0,107,0,0
1,105,4,1,0,2,2,4,2,0,1,22,55,0,0
2,89,1,3,0,1,2,2,1,1,0,2,72,0,0
3,132,4,0,0,0,5,0,1,0,0,19,85,0,0
4,115,2,6,0,1,0,1,1,0,0,17,82,0,0


In [ ]:
y = df['Label']
y.head()

,Label
0,0
1,0
2,0
3,0
4,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print(len(X_train))
print(len(X_test))

8000
2000


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = LogisticRegression(max_iter=2000,class_weight='balanced')
model.fit(X_train_scaled,y_train)

LogisticRegression(class_weight='balanced', max_iter=2000)

In [ ]:
y_pred = model.predict(X_test_scaled)
print(f"Accuracy: {accuracy_score(y_test,y_pred)*100: .2f}%")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nDetailed Report:")
print(classification_report(y_test, y_pred))

Accuracy:  90.70%

Confusion Matrix:
[[886 126]
 [ 60 928]]

Detailed Report:
              precision    recall  f1-score   support

           0       0.94      0.88      0.91      1012
           1       0.88      0.94      0.91       988

    accuracy                           0.91      2000
   macro avg       0.91      0.91      0.91      2000
weighted avg       0.91      0.91      0.91      2000



In [ ]:
test_url = "http://google.com/login/secure?user=123"
print(featureExtraction(test_url))

[39, 1, 0, 0, 1, 0, 1, 1, 0, 0, 3, 28, 0, 0]


In [ ]:
def predict_url(url, model, scaler):

    features = featureExtraction(url)

    df_features = pd.DataFrame([features], columns=X.columns)

    scaled_features = scaler.transform(df_features)

    prediction = model.predict(scaled_features)[0]
    probability = model.predict_proba(scaled_features)[0][1]

    return prediction, probability

In [ ]:
url = "https://gemini.google.com/app/395c80495bbdef25"
pred, prob = predict_url(url, model, scaler)

print(f"URL: {url}")
print(f"Prediction: {'PHISHING' if pred == 1 else 'SAFE'}")
print(f"Probability: {prob*100:.2f}% Phishing\n")


URL: https://gemini.google.com/app/395c80495bbdef25
Prediction: PHISHING
Probability: 98.03% Phishing

